# Start DASK Client

In [4]:
from dask.distributed import Client

In [5]:
option_comp = 'local'
if option_comp == "merlin":
    from dask_jobqueue import SLURMCluster
    cluster = SLURMCluster(cores     = 4, 
                           memory    ='4GB', 
                           walltime  = '10:00:00',
                           interface ='ib0',
                           local_directory = '/data/user/kim_a',
                           log_directory   = '/data/user/kim_a',
                          ) 
    
elif option_comp == "local":
    from dask.distributed import LocalCluster
    cluster = LocalCluster()

In [6]:
client = Client(cluster)

In [7]:
n_workers = 4
cluster.scale(n_workers)

In [8]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49670,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:49688,Total threads: 3
Dashboard: http://127.0.0.1:49691/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:49674,


In [ ]:
# client.close()
# cluster.close()

# Postprocessing of model outputs Y and saving

In [9]:
import bw2data as bd
import numpy as np
import pickle
import dask
from pathlib import Path

# Local files
from gsa_geothermal.global_sensitivity_analysis import task_per_worker

if __name__ == '__main__':
    project = 'Geothermal'
    bd.projects.set_current(project)

    option = 'conventional'
    diff_distr = True  # set to true when checking for robustness of GSA results to distribution choice
    iterations = 16  # number of Monte Carlo iterations per model input

    # Directory for saving results
    write_dir = Path("write_files")
    option_dir = "{}_N{}".format(option, iterations)
    if diff_distr:
        option_dir = "{}_robust".format(option_dir)
    write_dir_option = write_dir / option_dir / "scores"
    write_dir_option.mkdir(parents=True, exist_ok=True)

    

In [2]:
# %%time
# # Test
# n_workers = 25
# i_worker = 5
# task_per_worker(project, iterations, option, n_workers, i_worker, write_dir_option, diff_distr=diff_distr)

/Users/akim/opt/miniconda3/envs/geothermal-dev-py39/lib/python3.9/site-packages/SALib/sample/saltelli.py:94: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (5) is equal to `2^n`.
        
  warnings.warn(msg)


CPU times: user 25 s, sys: 1.27 s, total: 26.3 s
Wall time: 18.1 s


array([[5.64008483e-02, 1.09588528e-04, 8.22427039e-02, 5.91088134e-07,
        3.91234234e-05, 4.25276760e-04, 2.87124333e-09, 4.94295648e-04,
        7.74657945e-09, 1.58311679e-09, 1.23433037e-04, 7.34211783e-10,
        3.75743071e-03, 1.81104088e-01, 5.61401183e-02, 2.48566163e-07],
       [5.46835997e-02, 9.41015764e-05, 6.98577840e-02, 5.14374231e-07,
        3.32136180e-05, 3.61107698e-04, 2.48052713e-09, 4.24522213e-04,
        6.68049033e-09, 1.35305483e-09, 1.05158165e-04, 6.47594086e-10,
        3.30757883e-03, 1.56925653e-01, 4.97970009e-02, 2.23029550e-07],
       [5.62447350e-02, 1.08180626e-04, 8.11168021e-02, 5.84114148e-07,
        3.85861692e-05, 4.19443218e-04, 2.83572370e-09, 4.87952618e-04,
        7.64966231e-09, 1.56220210e-09, 1.21771688e-04, 7.26337459e-10,
        3.71653514e-03, 1.78906052e-01, 5.55634720e-02, 2.46244656e-07]])

In [10]:
task_per_worker = dask.delayed(task_per_worker)

model_evals = []
for i in range(n_workers):
    model_eval = task_per_worker(project, iterations, option, n_workers, i, write_dir_option, diff_distr=diff_distr)
    model_evals.append(model_eval)

In [11]:
%%time
Y_intermediate = dask.compute(model_evals)  

CPU times: user 18.5 s, sys: 4.57 s, total: 23.1 s
Wall time: 9min 7s


In [ ]:
Y_intermediate = np.array(Y_intermediate).squeeze()
Y_all_methods = np.vstack(Y_intermediate)

filepath_Y = write_dir_option / 'all_scores.pickle'
with open(filepath_Y, 'wb') as f:
    pickle.dump(Y_all_methods, f)
